# YouTube Script Preprocessing

- v1 Problem : 한 글자 및 추가 불용어 제거 안 되는 오류
- v2 Solution : 함수 분할 및 파이프라인 설계 -> 오류 수정 실패

- EDA 필요  
1. 채널별 업로드 현황 (1일 업로드 수, 종목별 업로드 수)
2. Text 전처리 전후 len() 비교

- Questions :  
1. 특정 날짜 기준으로 텍스트 데이터 통합하여 하나의 list로 생성한 후 분석  
2. 데이터프레임 유지한 상태로 분석 후 결과(ex. 빈도 for 워드 클라우드)만 취합

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# DB 연결
import pymysql

# 실시간 주식가격 데이터
import FinanceDataReader as fdr

# 텍스트 분석
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from konlpy.tag import Okt
from gensim import models

from wordcloud import WordCloud

# 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

# 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

# * 함수 선언

## (1) set_data() - YouTube Data version 수정

In [2]:
# 출처 '자체제작 감성어휘사전 모듈_ver2.ipynb'

########## 변경 사항 ##########
## ['date_h'] 생성 하지 않음
## 날짜 기준 내림차순으로 정렬 ??
## 15시 이후 날짜 조정 없음
## print 추가 등


def set_data(df,com_num = 0):  # panda, numpy, datetime, FinanceDataReader, konlpy, Counter
    # 일, 시간 나누기
    df['date'] = df['date'].astype(str)
    df['date_d'] = df['date'].str[:-2]
##     df['date_h'] = df['date'].str[-2:]
    # 타입을 데이트 타입으로 만듬
    df['date_d'] = pd.to_datetime(df['date_d'])
    
## 내림차순으로 변경 안 됨 => 해결 못 함
    df = df.sort_values(by='date_d') # 일기준으로 오름차순 정렬 
    
    if com_num != 0:
    #     통합 데이터 활용시 업종코드 지정
        df = df[df['st_cd'] == com_num] # 해당 회사만 추출
        num = str(com_num).zfill(6) # 종목코드를 6자리로 만들어줌
        
    else:
        # 특정 업종만 할 때
        num = str(df['st_cd'].iloc[0]).zfill(6)
    
    # 널값 제거
    df = df.dropna()

## 날짜 조정 불필요
#     # ============== 날짜 조정
#     ## 전일 15시 ~ 금일 15시로 날짜 조정
#     after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']
    
#     for i in range(len(df)):
#         if df['date_h'].iloc[i] in after_market:
#             df['date_d'].iloc[i] += datetime.timedelta(1)
    
    # 주말 및 공휴일 데이터
    ### Holidays
    try:
        # 서버가 열려있을 때
        db = pymysql.connect(user='root',
                             passwd='1234',
                             host='3.35.70.166',
                             db='proj',
                             charset='utf8')

        cursor = db.cursor(pymysql.cursors.DictCursor)

        sql = "select * from holidays"
        cursor.execute(sql)
        result = cursor.fetchall()
    
        # DataFrame으로 변경
        holi = pd.DataFrame(result)
        # db 닫기 --> 안하면 메모리 잡아먹음
        db.close()
    except:
        # 서버 없을 때 깃허브에서 바로 가져옴
        db_holi = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/%EC%97%B0%EC%8A%B5%20%EB%8D%B0%EC%9D%B4%ED%84%B0/holidays.csv'
        holi = pd.read_csv(db_holi)
    
    # date 컬럼을 날짜 형식으로 변경
    holi['date'] = pd.to_datetime(holi['date'])
    
    
    ### ===================주말 및 공휴일 제외
    
    ## 주말 및 공휴일만 추출
    market_closed = holi[holi['holiday']=="O"].reset_index(drop=True)
    
    ## 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    # 주말 및 공휴일 제외
    while len(df[df['date_d'].isin(market_closed_list)]['date_d']) !=0:
        for i in df[df['date_d'].isin(market_closed_list)]['date_d'].index:
            df['date_d'][i] += datetime.timedelta(1)
     
    
    
    # 주식 데이터 가져오기
    start_date = str(df['date_d'].iloc[0])[:10].replace('-', '')
    end_date = str(df['date_d'].iloc[-1])[:10].replace('-', '')  # 뉴스 데이터 기준 최근 값까지의 주식 데이터
    
    stock = fdr.DataReader(num, start = start_date, end = end_date)
    
    # change 컬럼 판단 기준 close에서 수수료 보다 많은 비율이 올랐으면 1, 떨어졌으면 -1, 그 사이 0
    point = stock['Close'] * stock['Change']
    spli = stock['Close'] * 0.001  # 수수료가 0.1 % 라고 할 때
    
    stock['ud'] = np.where( point > spli, 1,
                          np.where(point < spli, -1, 0))
    
    stock = stock.reset_index() # 날짜 컬럼 가져오기
    
    s_1 = stock[['Date','ud']] # 필요한 컬럼만 가져오기
    
    s_1.columns = ['date_d','ud'] # 컬럼 명 초기화
    
    # 뉴스데이터와 주식데이터 merge
    df_1 = pd.merge(df,s_1,on='date_d')  # 영향받는 컬럼 기준으로 inner 조인 
    
## 날짜순 정렬 (위에서 내림차순 정렬 안 됨)
#     df_1 = df_1.sort_values(by=['date_d'], axis=0, ascending=False)
    
## print 추가
    print('데이터 셋 구성 완료')
    
    return df_1

### set_data() TEST

In [10]:
# dataSet = set_data(df_lgc)

데이터 셋 구성 완료


In [11]:
# dataSet.head()

,st_n,st_cd,ch_nm,date,title,text,views,length,date_d,date_h,ud
0,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염...,자 그러면 오늘 또 증시 개장 상 1정리 해 주시아 염수 아니라 니모 지겠습니다 어...,100475,1813,2021-09-08,00,-1
1,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자 오늘 9월 8일 수요일 순서와먼저 뉴스 3부터 내 무게 좀 정리를해 드릴 텐데 ...,15483,1348,2021-09-08,00,-1
2,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[글로벌마켓 브리핑] 스가 총리 퇴진 발표 이후, 일본 증시 상승 랠리! f. 미래...",여러분 안녕하세요 광고 보고 왔습니다오늘 힘차게 한번 시작해 보도록하겠습니다 언제나...,7231,1711,2021-09-08,00,-1
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4 아 오늘한국투자증권 에서 오셨죠 우리박지영 차장을 모시겠습니다 어서오십시오 네 ...,21472,1511,2021-09-08,00,-1
4,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4 아 오늘한국투자증권 에서 오셨죠 우리박지영 차장을 모시겠습니다 어서오십시오 네 ...,21472,1511,2021-09-08,00,-1


In [21]:
# dataSet.title[0]

'[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염승환 이사'

In [22]:
# len(dataSet.text[0])

12909

In [23]:
# dataSet.text[0]

'자 그러면 오늘 또 증시 개장 상 1정리 해 주시아 염수 아니라 니모 지겠습니다 어서오십시오 네마 같습니까자 우리 여기서 님 저희가 한번 무슨얘기를 하든 걔도 모습은 앞에 이런모습 그 일로 모습 모집하고 있구요밑에도 노래 아이오와 따가 못 해마다픽으로 달라는 겁니다 우리도 그위험부담은 못하니까 l 합니다 이렇게때 못하며 예맥 에 강한 분들도 좀 요청해 주시기바라겠습니다 아김택진 진영이와 comte 머신 너무짜다예매트 뭐 저희가 참고하겠습니다 예 자우리 어머님과 함께 오늘도 증시 개장상황을 살펴 볼 텐데 오늘 한 10분정도 남았구요 먼저 그러면 어 리프트들 좀 보고 넘어갈까요 4으첫번째로는 요새 문은 행주 들도 좀많이 관심 들은 없긴한데 크긴 크지만좋고 일단 하나금융의 재정 옥애널리스트 그 은행 업종을 좀 되게간격으로 보고 있는 것 같아요 내간단히 말씀드리면 지금 6월 이후로그 미국의 10년 금리가 좀 빠지면서은행 제도 정확하게 동행하면서좀 주가가 부진하다 이렇게 좀 그최근 흐름을 좀 적어 줬고요 담아이제 긍정적으로 볼 부분들이 좀높아졌다 일단 크로나 금융지원 추가연장 일단 이렇게 졌기 때문에 추가충당금 적립 가능성 일단 조금낮아졌다고 합니다 그리고 3개월 동안외국인 순매도 가 계속 좀 이어졌는데일단 이게 멈췄고한 행 이제 코스피 200 때문에은행들 좀 수급 무료가 좀 있었는데이것도 이제 이번주 목요일 지만 사실끝나고 요그 다음에 여기에 8월 기준금리 인상연내 추가 인상 가능성도 높아진상황이니까은행주 실적 컨센서스 는 계속 상황이될 수 있다 요런 좀 내용이고 또테이퍼링 시계도 좀 논란은 있지만연주 4 긴축 속도가 좀 만 하다면경기 확장 북면 은 더 연장될가능성이 있다 그래서 다시한번 좀경기와 금리의 베팅할 시간이 좀돌아왔다요즘 내용이고 그래서 실적 상황에따라서주 당 대 당 금도 올라갈 가능성이높아졌기 때문에 4장 투자 수요도증가할 가능성이 있다고 합니다 그래서일단 변경된 은행 그 전체순위 이게일단 18.5 좋은 정도로 작년 대비해서 5% 가 내년도 순위 이끌 

# * 데이터 로드

(1) DB에서 CSV 파일 다운로드  
(2) DB, Django 직접 접근?

## (1) DB에서 CSV 파일 다운로드  

In [3]:
## 출처 '데이터 csv파일로 저장.ipynb'

# com_list = {'하이닉스' : '000660',
#            '현대차' : '005380',
#            '삼성전자' : '005930',
#            'LG화학' : '051910',
#            '셀트리온' : '068270'}
# media_list = {'아시아경제_' : 'asia_',
#              '매일경제_': 'maeil_',
#              'youtube_한국경제_':'hk_',
#              'youtube_삼프로_':'sampro_',
#              'youtube_슈카_':'suka_'}

# for n,m in media_list.items():
#     for i,j in com_list.items():

#         db = pymysql.connect(
#             user='root', 
#             passwd='1234', 
#             host='3.35.70.166', 
#             db='proj', 
#             charset = 'utf8'
#         )

#         cursor = db.cursor(pymysql.cursors.DictCursor)
#         if 'youtube' in n:
#             sql = "select * from youtube_"+ m + j +" where length(date) = 10"
#         else:
#             sql = "select * from "+ m +"news_craw_" + j +" where length(date) = 10"  # date 널값 제외
#         cursor.execute(sql)

#         result = cursor.fetchall()

#         # 데이터 프레임으로 변경
#         df = pd.DataFrame(result)

#         # csv 저장
#         df.to_csv('df_'+ n + i + '.csv', index=False)

In [3]:
com_list = {'하이닉스' : '000660',
           '현대차' : '005380',
           '삼성전자' : '005930',
           'LG화학' : '051910',
           '셀트리온' : '068270'}

## media_list에서 youtube list만 가져오기
youtube_list = {'youtube_한국경제_':'hk_', 
                'youtube_삼프로_':'sampro_',
                'youtube_슈카_':'suka_'}

company_list = ['하이닉스', '현대차', '삼성전자', 'LG화학', '셀트리온']

In [12]:
## csv 데이터 로드 & 데이터프레임명 - shape 출력

for n,m in youtube_list.items():
    for i,j in com_list.items():
        if i == '하이닉스':
            globals()['{}skhynix'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}skhynix'.format(m), globals()['{}skhynix'.format(m)].shape)
            
        elif i == '현대차':
            globals()['{}hyundai'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}hyundai'.format(m), globals()['{}hyundai'.format(m)].shape)
            
        elif i == '삼성전자':
            globals()['{}samsung'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}samsung'.format(m), globals()['{}samsung'.format(m)].shape)
            
        elif i == 'LG화학':
            globals()['{}lgchem'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}lgchem'.format(m), globals()['{}lgchem'.format(m)].shape)
            
        else:
            globals()['{}celltrion'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}celltrion'.format(m), globals()['{}celltrion'.format(m)].shape)
            print()

hk_skhynix (504, 10)
hk_hyundai (526, 10)
hk_samsung (546, 10)
hk_lgchem (500, 10)
hk_celltrion (484, 10)

sampro_skhynix (489, 10)
sampro_hyundai (526, 10)
sampro_samsung (534, 10)
sampro_lgchem (506, 10)
sampro_celltrion (460, 10)

suka_skhynix (90, 10)
suka_hyundai (403, 10)
suka_samsung (448, 10)
suka_lgchem (64, 10)
suka_celltrion (197, 10)



### 데이터프레임 열 생략

In [3]:
# description, url 제외
# sampro_lgchem = sampro_lgchem.iloc[:, :8]

In [4]:
# sampro_lgchem.head()

,st_n,st_cd,ch_nm,date,title,text,views,length
0,LG화학,51910,삼프로TV_경제의신과함께,2021090900,[증시 염탐정] 출발 분위기 안좋은 시장.. 삼성전자 흐름을 주목해보자 f.이베스트...,자 그러면 어저희와 함께 오늘 증시 개장 상황 이살펴 주실 염 블리 e 베스트투자증...,42597,1826
1,LG화학,51910,삼프로TV_경제의신과함께,2021090900,"[글로벌 이슈체크] 수요 꺾이지 않은 글로벌 배터리, 비중 확대해도 괜찮을까? f...",글로벌 iv 부 시작하도록 하겠습니다아니 오늘 및 미래 3 측에서새식구를 보내고 나...,17745,1556
2,LG화학,51910,삼프로TV_경제의신과함께,2021090900,"투자 전략 전문가가 바라보는 국내 증시 방향성은?_박병창, 한지영, 염승환",[음악]으no[음악]cool합니다[음악]아 으[음악]아으[박수][음악]기피하는 경제...,309214,7095
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자 오늘 9월 8일 수요일 순서와먼저 뉴스 3부터 내 무게 좀 정리를해 드릴 텐데 ...,15483,1348
4,LG화학,51910,삼프로TV_경제의신과함께,2021090900,[심층 인터뷰] 추석과 FOMC.. 포트폴리오의 조정이 필요하다? f.키움증권 한지...,자 그러면 오늘 아침 아 김 증권의한지영 아예 무영과 함께 우리전략을 한번 노래하고...,14930,1570


## (2) DB, Django 직접 접근?
- 데이터 엔지니어 영역(?)

---

# * 날짜별 데이터 분리

### 특정 날짜 데이터만 가지고 오기 ?? 
- '특정 날짜' & '모든 종목' 데이터 로드
- '특정 날짜' & '모든 유튜버' 데이터 로드

#### TEST 1
- LG화학 데이터만 이용해서 테스트 중  
LG 화학 - 삼프로tv - 2021.9.8일자

In [11]:
lgc['date_d'].value_counts()

2021-09-07    9
2021-09-09    7
2020-09-17    7
2021-01-04    6
2021-08-24    6
             ..
2020-12-17    1
2020-12-18    1
2020-12-21    1
2020-12-22    1
2021-09-15    1
Name: date_d, Length: 252, dtype: int64

In [17]:
lgc['date_d'].value_counts().describe()

count    252.000000
mean       2.007937
std        1.350792
min        1.000000
25%        1.000000
50%        2.000000
75%        2.250000
max        9.000000
Name: date_d, dtype: float64

In [13]:
# 특정 월만 가져오기 ??
# str(lgc['date_d'][0])[:7]

'2019-09'

In [17]:
# 날짜 순 정렬 TEST => set_data 함수 
lgc = lgc.sort_values(by=['date_d'], axis=0, ascending=False)

In [18]:
# lg chemical [날짜] before

lgc_210914_raw = lgc[lgc['date_d'] == '2021-09-14']
print(lgc_210914_raw.shape)
lgc_210914_raw.head()

(5, 12)


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
504,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[퇴근길 라이브&백브리핑] 카카오 집어삼킨 금융소비자보호법, 지금 칼 빼든 이유는?...",[음악]으 으cool으[음악]으[음악][음악]투자를 하는 사람들의 깊이 있는데 와3...,188383,9580,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=0nsLNwKL40U,2021-09-14,-1
503,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[글로벌마켓 브리핑] 중국 압박하는 미국정부, 정상간 통화의 기대가 실망으로 f. ...",4 실시간 티칭 창을 봤더니 우리주님이 아침에 오라고 해서 표정 동안이 어떻게 이렇...,10914,1730,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=wJz_JpmgTO4,2021-09-14,-1
502,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 염탐정] 양 시장 강세 출발.. 들어오는 외국인, 시장 안정되나? f.이베스...",이 황구 보시고 오셨습니다 오늘도언제나처럼 우리 증지 엄 탐정 1특수 1권의 염수 ...,39754,1537,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=XoLozRQFmdo,2021-09-14,-1
501,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"우주산업의 혁신, 이제 민간이 주도한다_글로벌 라이브 _글로벌 라이브_ 장의성, 서...",[음악][음악]으으으222[음악]글로벌 라이브 에 오신걸 환영합니다안녕하십니까 박종...,86620,6295,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=8L5755txAYs,2021-09-14,-1
500,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 셔터맨] 쏠림이 심한 국내 주식, 중심 잃지 않는 종목 선정이 필요하다. ...",4아침에 도착한 말씀드렸습니다만u 머리가 이상해 보인다면여러분의 패션 감각을 의심해...,21914,1573,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=JOK1oGTKi1U,2021-09-14,-1


#### TEST 2
- 셀트리온 - 슈카월드

In [16]:
suka_cell = suka_celltrion.head()

In [18]:
# suka_cell_sd = set_data(suka_cell)

# * (2) text_pre() & stop_words() 

## youtube_pre 로 합쳐서 만들기(예정)

In [19]:
# 출처 '자체제작 감성어휘사전 모듈_ver2.ipynb' > def giro_dic
##표시 수정 부분

def text_pre(df,num = 0):
        
## pipeline으로 이동 => 실패
    df = set_data(df, num)
#     print('데이터 셋 구성 완료')
    
    okt = Okt()
    print('okt 모듈 활성')
    
    
    # 데이터 전처리
    # =================== nlp
    # 1. 제목, 텍스트 --> 명사, 형용사 추출
    # 태그 삭제        
    # \n \t \s 삭제
## print문 추가1
    print('태그 삭제 중')
    df['text'] = df['text'].str.replace('[\n|\t|\r|\s]','')
    df['text'] = df['text'].str.replace('[^a-zA-Z0-9가-힣]','')

## print문 추가2
    print('품사 태깅 작업 중')    
    # 토큰화 및 품사 태깅
    df['Contents_input'] = df['text'].apply(lambda x: okt.pos(x, norm=True, stem=True))
    print('태그 삭제 및 품사 태깅 완료')
    
    # 명사, 형용사 추출
    for i in range(len(df['Contents_input'])):
        wordList = []
        for j in range(len(df['Contents_input'][i])):
            if df['Contents_input'][i][j][1] == 'Noun' or df['Contents_input'][i][j][1] == 'Adjective':
                wordList.append(df['Contents_input'][i][j][0])         
        df['Contents_input'][i] = wordList
    print('명사, 형용사 추출')
    
## print문 추가3
    print('text_pre 작업 완료!')
    
    return df

In [ ]:
''' # 추후 미디어별 업데이트
media = ['오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
         '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
         '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
         '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']

## 함수 내 media 전역 변수 선언
#     global media
'''

In [27]:
def stop_words(df,num = 0):
    
## media 전역 변수 선언
#     global media
    
    # 2. 불용어 처리
    # 불용어 사전 로딩
##     url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/stopwords_ver1.txt'
    url = 'https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])
##     media = ['매일경제','mkcokr','무단전재및재배포금지','무단전재','재배포금지']
    media = ['실시간', '오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
             '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
             '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
             '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']
    print('불용어 사전 로딩')
    
    
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in stopwords:
                df['Contents_input'][i].remove(word)
    print('불용어 1차 처리 완료')
    
## for문 분리                
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in media:
                df['Contents_input'][i].remove(word) 
    print('불용어 2차 처리 완료')
    

## 단어 길이 1 이하 제거

    # (1) 단어길이가 1이하인 단어 제거
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        for word in wordList:
            if len(word) <= 1:
                df['Contents_input'][index].remove(word)
    print('단어 길이 1이하인 단어 제거')

    ''' TEST 해보기!
    # (2) 단어 길이가 1 초과인 단어
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        wordList = [x for x in wordList if len(x) > 1]
    print('단어 길이 2이상인 단어 리스트 생성')    
    '''

## print문 추가4
    print('stop_words 작업 완료!')
    
    return df

In [ ]:
''' from 형림님
hangeul = re.compile('[^ ㄱ-ㅣ 가-힣]')                         # 정규 표현식  → 한글 추출 규칙 : 띄어쓰기(1개)를 포함한 한글
result = hangeul.sub('', globals()[corp_list[i]]['text'][j])   # 위에 설정한 hangeul 규칙을 text에 적용
okt = Okt()                                                    # 형태소 추출
nouns = okt.nouns(result)
nouns = [x for x in nouns if len(x) > 1]                       # 한 글자 키워드 제거
nouns = [x for x in nouns if x not in stopwords]               # 불용어 제거
            
corpus = " ".join(nouns)                                       # List를 String으로 변환
globals()[corp_list[i]]['except_stopwords'][j] = corpus
'''

## text_pre() & stop_words() TEST

### Test 0

In [141]:
## test

print(lgc_210908['Contents_input'][2][:20])

for i in range(len(lgc_210908['Contents_input'])):
    
    media = ['오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
             '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
             '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
             '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']

    for word in lgc_210908['Contents_input'][i]:
#             if word in stopwords:
#                 lgc_210908['Contents_input'][i].remove(word)
        if word in media:
            lgc_210908['Contents_input'][i].remove(word)
                
print(lgc_210908['Contents_input'][2][:20])

['광고', '힘차다', '시작', '증권', '의장', '선임', '전이', '매년', '안녕하세요우', '난수', '간이', '럼또뭡', '사과', '감히', '방소', '점점', '덥다', '고합', '다정', '말로']
['광고', '힘차다', '시작', '증권', '의장', '선임', '전이', '매년', '난수', '간이', '럼또뭡', '사과', '감히', '방소', '점점', '덥다', '고합', '다정', '말로', '사과']


### Test 1

In [ ]:
lgc_210908_tp = text_pre(lgc_210908_raw)
lgc_210908_tp.head()

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,date_h,ud_x,ud_y,Contents_input
0,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염...,자그러면오늘또증시개장상1정리해주시아염수아니라니모지겠습니다어서오십시오네마같습니까자우리...,100475,1813,삼프로TV제작사및멤버사공개채용세부공고확인링크httpstlySR1n쿨썸머여름특가반값세...,https://www.youtube.com/watch?v=Du9Anki-rDI,2021-09-08,00,-1,-1,"[자, 오늘, 또, 증시, 개장, 정리, 염수, 아니다, 니모, 어서, 오십, 시오..."
1,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자오늘9월8일수요일순서와먼저뉴스3부터내무게좀정리를해드릴텐데예아무래도미국소식도좀갤주너...,15483,1348,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=4HZCWvZpp6U,2021-09-08,00,-1,-1,"[자, 오늘, 부터, 무게, 좀, 정리, 예, 미국, 소식, 좀, 주너, 미국, 시..."
2,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[글로벌마켓 브리핑] 스가 총리 퇴진 발표 이후, 일본 증시 상승 랠리! f. 미래...",여러분안녕하세요광고보고왔습니다오늘힘차게한번시작해보도록하겠습니다언제나처럼미래에셋증권의...,7231,1711,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=IvdD2QdNO2E,2021-09-08,00,-1,-1,"[여러분, 안녕하다, 광고, 오늘, 힘차다, 한번, 시작, 처럼, 미래에셋, 증권,..."
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4아오늘한국투자증권에서오셨죠우리박지영차장을모시겠습니다어서오십시오네안녕하십니까4한국투...,21472,1511,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=Z04S7n9K74w,2021-09-08,00,-1,-1,"[늘, 한국, 투자, 증권, 우리, 박지영, 차장, 어서, 시오, 안녕하십니까, 한..."
4,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4아오늘한국투자증권에서오셨죠우리박지영차장을모시겠습니다어서오십시오네안녕하십니까4한국투...,21472,1511,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=Z04S7n9K74w,2021-09-08,00,-1,-1,"[늘, 한국, 투자, 증권, 우리, 박지영, 차장, 어서, 시오, 안녕하십니까, 한..."


In [34]:
lgc_210908 = stop_words(lgc_210908_tp)

불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
단어 길이 1이하인 단어 제거
stop_words 작업 완료


In [35]:
lgc_210908.head()

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,date_h,ud_x,ud_y,Contents_input
0,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 염탐정] 약보합으로 출발한 시장.. 대형주의 방향은? f.이베스트투자증권 염...,자그러면오늘또증시개장상1정리해주시아염수아니라니모지겠습니다어서오십시오네마같습니까자우리...,100475,1813,삼프로TV제작사및멤버사공개채용세부공고확인링크httpstlySR1n쿨썸머여름특가반값세...,https://www.youtube.com/watch?v=Du9Anki-rDI,2021-09-08,00,-1,-1,"[정리, 염수, 니모, 네마, 하든, 일로, 모집, 노래, 아이오와, 따갑다, 달라..."
1,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[뉴스3] ""美증시 비중축소""…잇단 비관론에 다우 270p↓",자오늘9월8일수요일순서와먼저뉴스3부터내무게좀정리를해드릴텐데예아무래도미국소식도좀갤주너...,15483,1348,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=4HZCWvZpp6U,2021-09-08,00,-1,-1,"[무게, 정리, 미국, 소식, 주너, 미국, 시장, 음소, 간밤, 진행, 좋다, 아..."
2,LG화학,51910,삼프로TV_경제의신과함께,2021090800,"[글로벌마켓 브리핑] 스가 총리 퇴진 발표 이후, 일본 증시 상승 랠리! f. 미래...",여러분안녕하세요광고보고왔습니다오늘힘차게한번시작해보도록하겠습니다언제나처럼미래에셋증권의...,7231,1711,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=IvdD2QdNO2E,2021-09-08,00,-1,-1,"[광고, 힘차다, 시작, 증권, 의장, 선임, 전이, 매년, 난수, 간이, 럼또뭡,..."
3,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4아오늘한국투자증권에서오셨죠우리박지영차장을모시겠습니다어서오십시오네안녕하십니까4한국투...,21472,1511,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=Z04S7n9K74w,2021-09-08,00,-1,-1,"[한국, 투자, 증권, 차장, 한국, 투자, 증권, 연금, 또또, 보관, 수수료, ..."
4,LG화학,51910,삼프로TV_경제의신과함께,2021090800,[증시 셔터맨] 외인과 기관은 왜 삼성전자와 카카오뱅크를 매도할까? f. 한국투자증...,4아오늘한국투자증권에서오셨죠우리박지영차장을모시겠습니다어서오십시오네안녕하십니까4한국투...,21472,1511,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=Z04S7n9K74w,2021-09-08,00,-1,-1,"[한국, 투자, 증권, 차장, 한국, 투자, 증권, 연금, 또또, 보관, 수수료, ..."


In [38]:
lgc_210908['Contents_input'][2]

['광고',
 '힘차다',
 '시작',
 '증권',
 '의장',
 '선임',
 '전이',
 '매년',
 '난수',
 '간이',
 '럼또뭡',
 '사과',
 '감히',
 '방소',
 '점점',
 '덥다',
 '고합',
 '다정',
 '말로',
 '사과',
 '주말',
 '생각',
 '한국',
 '시장',
 '좋다',
 '중국',
 '뜨겁다',
 '언니',
 '중국',
 '뜨겁다',
 '일본도',
 '뜨겁다',
 '일본도',
 '안되다',
 '고열',
 '어보',
 '기대하다',
 '전혀',
 '매력',
 '이구',
 '의하다',
 '중국',
 '걱정',
 '면서',
 '중국',
 '시장',
 '사고',
 '안치',
 '중국',
 '시장',
 '당연하다',
 '파란색',
 '뜨겁다',
 '중국',
 '시장',
 '말씀',
 '상해',
 '종합',
 '지수',
 '가요',
 '연중',
 '최고점',
 '돌파',
 '고요',
 '영어',
 '문제',
 '여년',
 '최고점',
 '이요',
 '종가',
 '포인트',
 '올해',
 '이란',
 '포인트',
 '도제',
 '어제',
 '그거',
 '미타',
 '안팎',
 '상황',
 '빨갛다',
 '업계',
 '중국',
 '정부',
 '반토막',
 '연중',
 '최고점',
 '종합',
 '지수',
 '지난주',
 '느가파르',
 '먼저',
 '나머지',
 '심천',
 '지불',
 '홍콩',
 '항셍지수',
 '판지',
 '주말',
 '고생',
 '어제',
 '또고',
 '은낙폭',
 '대비',
 '반등',
 '바이오',
 '인프라',
 '투자',
 '모델',
 '원자재',
 '가격',
 '대해',
 '가격',
 '덥다',
 '시크릿',
 '과업',
 '좋다',
 '중국',
 '뭐호산',
 '철강',
 '차트',
 '보시',
 '난리',
 '도아',
 '고또',
 '최근',
 '제사',
 '각지',
 '국영',
 '기업',
 '전투',
 '국영',
 '기업',
 '이또',
 '사회',
 '장의',
 '상상',
 '바닥',


### Test 2

In [28]:
%%time
lgc_210914_tp = text_pre(lgc_210914_raw)

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 중
품사 태깅 작업 중
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
text_pre 작업 완료!
Wall time: 5min 53s


In [22]:
lgc_210914_tp.head()

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input
0,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[퇴근길 라이브&백브리핑] 카카오 집어삼킨 금융소비자보호법, 지금 칼 빼든 이유는?...",음악으으cool으음악으음악음악투자를하는사람들의깊이있는데와32tv저녁라이브9월13일순...,188383,9580,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=0nsLNwKL40U,2021-09-14,-1,-1,"[음악, 음악, 투자, 사람, 깊이, 있다, 저녁, 라이브, 위염, 수관, 님, 오..."
1,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[글로벌마켓 브리핑] 중국 압박하는 미국정부, 정상간 통화의 기대가 실망으로 f. ...",4실시간티칭창을봤더니우리주님이아침에오라고해서표정동안이어떻게이렇게정확하게아시죠아니밤늦...,10914,1730,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=wJz_JpmgTO4,2021-09-14,-1,-1,"[실시간, 티칭창, 우리, 주님, 아침, 오라, 고해, 표정, 안이, 어떻다, 정확..."
2,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 염탐정] 양 시장 강세 출발.. 들어오는 외국인, 시장 안정되나? f.이베스...",이황구보시고오셨습니다오늘도언제나처럼우리증지엄탐정1특수1권의염수만이사님다올계십니다이사...,39754,1537,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=XoLozRQFmdo,2021-09-14,-1,-1,"[황구, 오늘, 처럼, 우리, 증, 지엄, 탐정, 특수, 권, 염수, 만이사, 이사..."
3,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"우주산업의 혁신, 이제 민간이 주도한다_글로벌 라이브 _글로벌 라이브_ 장의성, 서...",음악음악으으으222음악글로벌라이브에오신걸환영합니다안녕하십니까박종우입니다자주식시장이런...,86620,6295,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=8L5755txAYs,2021-09-14,-1,-1,"[음악, 음악, 음악, 글로벌, 라이브, 걸, 환영, 안녕하십니까, 박종우, 이다,..."
4,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 셔터맨] 쏠림이 심한 국내 주식, 중심 잃지 않는 종목 선정이 필요하다. ...",4아침에도착한말씀드렸습니다만u머리가이상해보인다면여러분의패션감각을의심해보셔야됩니다청담...,21914,1573,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=JOK1oGTKi1U,2021-09-14,-1,-1,"[아침, 착하다, 머리, 이상하다, 여러분, 패션, 감각, 의심, 청담, 봉인, 책..."


In [29]:
%%time
lgc_210914 = stop_words(lgc_210914_tp)

불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
단어 길이 1이하인 단어 제거
stop_words 작업 완료!
Wall time: 1.83 s


In [25]:
lgc_210914.head()

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input
0,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[퇴근길 라이브&백브리핑] 카카오 집어삼킨 금융소비자보호법, 지금 칼 빼든 이유는?...",음악으으cool으음악으음악음악투자를하는사람들의깊이있는데와32tv저녁라이브9월13일순...,188383,9580,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=0nsLNwKL40U,2021-09-14,-1,-1,"[투자, 사람, 깊이, 저녁, 라이브, 위염, 수관, 저녁, 뭐또, 어류, 저녁, ..."
1,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[글로벌마켓 브리핑] 중국 압박하는 미국정부, 정상간 통화의 기대가 실망으로 f. ...",4실시간티칭창을봤더니우리주님이아침에오라고해서표정동안이어떻게이렇게정확하게아시죠아니밤늦...,10914,1730,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=wJz_JpmgTO4,2021-09-14,-1,-1,"[실시간, 티칭창, 주님, 아침, 오라, 고해, 표정, 안이, 정확하다, 좋다, 밤..."
2,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 염탐정] 양 시장 강세 출발.. 들어오는 외국인, 시장 안정되나? f.이베스...",이황구보시고오셨습니다오늘도언제나처럼우리증지엄탐정1특수1권의염수만이사님다올계십니다이사...,39754,1537,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=XoLozRQFmdo,2021-09-14,-1,-1,"[황구, 처럼, 지엄, 탐정, 특수, 염수, 만이사, 이사, 안녕하다, 유지은, 코..."
3,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"우주산업의 혁신, 이제 민간이 주도한다_글로벌 라이브 _글로벌 라이브_ 장의성, 서...",음악음악으으으222음악글로벌라이브에오신걸환영합니다안녕하십니까박종우입니다자주식시장이런...,86620,6295,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=8L5755txAYs,2021-09-14,-1,-1,"[글로벌, 라이브, 환영, 박종우, 이다, 주식시장, 많다, 사면, 계시다, 사면,..."
4,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 셔터맨] 쏠림이 심한 국내 주식, 중심 잃지 않는 종목 선정이 필요하다. ...",4아침에도착한말씀드렸습니다만u머리가이상해보인다면여러분의패션감각을의심해보셔야됩니다청담...,21914,1573,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=JOK1oGTKi1U,2021-09-14,-1,-1,"[아침, 착하다, 머리, 이상하다, 패션, 감각, 의심, 청담, 봉인, 책보, 청담..."


In [30]:
lgc_210914['Contents_input'][1]

['티칭창',
 '주님',
 '아침',
 '오라',
 '고해',
 '표정',
 '안이',
 '정확하다',
 '좋다',
 '밤늦다',
 '게일',
 '사람',
 '새벽',
 '가혹하다',
 '말씀',
 '프로',
 '형님',
 '아예',
 '아침',
 '망각',
 '미국',
 '어린이',
 '엄마',
 '교수',
 '를왜',
 '요약',
 '간각',
 '나니',
 '라리',
 '마음',
 '맺음말',
 '데이',
 '생각',
 '유모',
 '프린트',
 '손발',
 '사회',
 '고민',
 '주도',
 '국위',
 '들것',
 '국민',
 '아예',
 '방송',
 '배탓',
 '해나',
 '오니',
 '간단하다',
 '애플',
 '금요일',
 '매달',
 '좋다',
 '에픽',
 '게임',
 '독점',
 '소송',
 '요새',
 '애플',
 '버스',
 '차장',
 '댓글',
 '정답',
 '계시다',
 '마차',
 '추가',
 '막차',
 '길가',
 '테팅닭',
 '은근',
 '짜증',
 '내지',
 '가끔',
 '아크',
 '예금',
 '게속',
 '또왜',
 '분위기',
 '의하다',
 '알겟습',
 '소식',
 '많다',
 '중국',
 '포인트',
 '란말',
 '연중',
 '증식',
 '게뭐',
 '지난주',
 '계속',
 '여전하다',
 '좋다',
 '은또',
 '역시',
 '크게',
 '상관없다',
 '구경',
 '대시',
 '클리',
 '커리',
 '철강',
 '석탄',
 '화학',
 '알루미늄',
 '계속',
 '강세',
 '보이',
 '면서',
 '사회',
 '조합',
 '좋다',
 '수준',
 '바지',
 '심천',
 '성분',
 '창업',
 '하라',
 '관창',
 '타락',
 '어항',
 '생태',
 '일정',
 '이체',
 '퍼센트',
 '군내',
 '중국',
 '습하다',
 '화혼',
 '조세',
 '솔직하다',
 '걱정',
 '바쁘다',
 '사이',
 '확인',
 '게다',
 '자막',
 '장손',
 '매니저',
 '업데이트',
 

# * 파이프라인(보류)

In [56]:
# df_clean = youtube_pre(df)
# df_clean.head()

In [ ]:
df = text_pre(df)

df = stop_words(df)

In [ ]:
# ex. lgchem_210809_list

com_date_list = []

for i in com_date.Contents_input:
    for j in i:
        com_date_list.append(j)
        
count = Counter(com_date_list)
words = dict(count.most_common())

words_df = pd.DataFrame.from_dict([words]).T.reset_index()
words_df.columns = ['words', 'freq']

words_df.to_csv('com_date_wc.csv', header=True, index=False, encoding = 'utf-8-sig')